In [1]:
!pip install docarray

  Obtaining dependency information for docarray from https://files.pythonhosted.org/packages/fb/36/f671fa88de3d1cfb59cbc02659a13680f16fc0e9f6495453a068735baccd/docarray-0.37.1-py3-none-any.whl.metadata
  Obtaining dependency information for orjson>=3.8.2 from https://files.pythonhosted.org/packages/bc/57/17e518de07d0e2cae06259dd1676f589bc8873f1151b874b9e9d50cdbb13/orjson-3.9.5-cp39-cp39-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata
  Using cached orjson-3.9.5-cp39-cp39-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata (49 kB)
  Obtaining dependency information for rich>=13.1.0 from https://files.pythonhosted.org/packages/8d/5f/21a93b2ec205f4b79853ff6e838e3c99064d5dbe85ec6b05967506f14af0/rich-13.5.2-py3-none-any.whl.metadata
  Using cached rich-13.5.2-py3-none-any.whl.metadata (18 kB)
  Obtaining dependency information for types-requests>=2.28.11.6 from https://files.pythonhosted.org/packages/06/9b/04bb62f11a6824df5d4568439cf0715118c

In [2]:
import dotenv
import os
import openai

dotenv.load_dotenv()

# set up environment variables
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_API_BASE = os.environ['OPENAI_API_BASE']
OPENAI_API_TYPE = os.environ['OPENAI_API_TYPE']
OPENAI_API_VERSION = os.environ['OPENAI_API_VERSION']

# set up OpenAI client
openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_API_BASE
openai.api_type = OPENAI_API_TYPE
openai.api_version = OPENAI_API_VERSION

# Langchain for LLM App Development 

We talked about how building an LLM app involves doing some prompt management 
where we can either prepare the input data from the user with some 
pre-prompting, or do some post-prompting and some cleaning up after the LLM 
gives an output to ensure that our app performs the functionalities as expected.

So, this kind of workflow usually involves a lot of abstractions where prompts 
are no longer static pieces of text, but dynamic, they have to integrate 
information.

![](./images/Notebook_4-dynamic_prompt.png)

This dynamics requirement from a prompt will lead to the need for creating certain types of abstractions to properly handle and manage prompts effectively.

Another need in the context of more complex LLM App development, is the need for chaining prompts together, meaning connecting the output of one prompt to another. This is often the case for when prompts might be too large and a single call to the LLM won't be enough to solve the problem or the context window (maximum tokens/words the model can read and writer per request) is exceeded.

![](./images/Notebook_4-prompt_chaining.png)

# Lanchain

[Langchain](https://python.langchain.com/docs/get_started/introduction.html) is a framework created by Harrison Chase that facilitates the creation and management of dynamic prompts and chaining between prompts.

Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

With langchain it becomes much easier to create what are called Prompt Templates, which are prompts that can take in user data and abstract away the need for typing out everything that is required for a task to get done.

Let's take a look at some simple examples to get started.

In order to create an application with LangChain, we need to understand its core components:

- Models
- Prompts
- Indexes
- Chains
- Agents (won't go into it)

![](2023-08-17-14-48-39.png)

**Models**

abstractions over the LLM APIs like the ChatGPT API.​

In [6]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0, deployment_id="chatgpt-4")

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


You can predict outputs from both LLMs and ChatModels:

In [7]:
chat_model.predict("hi!")
# Output: "Hi"

'Hello! How can I assist you today?'

You can also use the predict method over a string input:

In [8]:
text = "What would be a good name for a dog that loves to nap??"


chat_model.predict(text)
# Output: "Snuggles"

'Snoozer'

Finally, you can use the `predict_messages` method over a list of messages:

In [9]:
from langchain.schema import HumanMessage

text = "What would be a good dog name for a dog that loves to nap?"
messages = [HumanMessage(content=text)]

chat_model.predict_messages(messages)


AIMessage(content='Snoozer', additional_kwargs={}, example=False)

**Prompts**

Prompt Templates are useful abstractions for reusing prompts. 

They are used to provide context for the specific task that the language model needs to complete. 
A simple example is a `PromptTemplate` that formats a string into a prompt:

In [10]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="nap")
# Output: "What is a good dog name for a dog that loves to nap?"

'What is a good dog name for a dog that loves to nap?'

# Chains

In [13]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=ChatOpenAI(deployment_id="chatgpt-4"),
    prompt=prompt,
)
chain.run("watch tennis on tv")

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


'Ace'

You can also create more complex ChatPromptTemplates that contains a list of ChatMessageTemplates:

In [14]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [15]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")
# Output: ['hi', 'bye']

['hi', 'bye']

**LLMChain**

Finally, you can combine all these components into an LLMChain:

In [16]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generated 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(engine="chatgpt-4"),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("dogs")
# Output: ['Golden Retriever','Labrador Retriever','German Shepherd','Bulldog','Poodle']

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


['Golden Retriever',
 'German Shepherd',
 'Labrador Retriever',
 'Beagle',
 'Bulldog']

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

One of the best applications of langchain is for the "chat with your data"-types of applications, where the user uploads a document like a pdf or a .txt file, and is able to query that document using langchain powered by an LLM like ChatGPT. 

# LangChain Lab Exercises

Let's take a look at a simple example of **Simple Sequential Chain**:

In [17]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain

In [18]:
# This is an LLMChain to write a synopsis given a title of a play.
llm = ChatOpenAI(temperature=.7, engine="chatgpt-4")
template = """You are a learning assistant. Given a technical subject, write down 5 fundamental concepts to understand it.
Subject: {subject}
Learning assistant: The 5 fundamental concepts are:"""
prompt_template = PromptTemplate(input_variables=["subject"], template=template)
learning_chain = LLMChain(llm=llm, prompt=prompt_template)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [19]:
# This is an LLMChain to write a review of a play given a synopsis.
#llm = ChatOpenAI(temperature=.7)
template = """You are an expert teacher in all technical and scientific fields. Given a list of 5 concepts, write down a simple intuitive explanation of each concept.
Concepts:
{concepts}
Intuitive explanations:"""
prompt_template = PromptTemplate(input_variables=["concepts"], template=template)
explanation_chain = LLMChain(llm=llm, prompt=prompt_template)

In [20]:
# This is the overall chain where we run these two chains in sequence.
learning_overall_chain = SimpleSequentialChain(chains=[learning_chain, explanation_chain], verbose=True)

In [21]:
output = learning_overall_chain.run("3D printing")
output



> Entering new SimpleSequentialChain chain...
1. Understanding of CAD (Computer-Aided Design): This involves the use of computer software to create 3D models. It's essential to know how to design or modify models for 3D printing.

2. Knowledge of 3D Printing Technologies: Different printing technologies such as FDM (Fused Deposition Modeling), SLA (Stereolithography), and SLS (Selective Laser Sintering) are used for different applications. Understanding how they work is crucial.

3. Materials Used in 3D Printing: Different materials such as plastics (ABS, PLA), metals, and ceramics are used in 3D printing. Knowing their properties and usage is important.

4. Slicing: This is the process of converting a 3D model into hundreds or thousands of horizontal layers. The slicing software then instructs the 3D printer to follow this blueprint to create the object layer by layer.

5. Post-Processing: After the printing process, the printed parts often need post-processing. This can include cle

"1. CAD (Computer-Aided Design): Imagine being a sculptor who is planning to make a sculpture. Instead of directly starting to chisel a stone, you decide to use a computer program to design your sculpture first. You can view it from all angles, make changes, and perfect your design before you even touch your stone. That's what CAD is - it's like your digital sketchpad where you can create, modify, and perfect your 3D model before you bring it into the real world through 3D printing.\n\n2. 3D Printing Technologies: Think of 3D printing technologies as different types of chefs. FDM is like a pastry chef who lays down layer after layer of dough to create a croissant. SLA is like a gourmet chef who uses a delicate process to create intricate dishes - it uses a laser to solidify liquid resin layer by layer. SLS, on the other hand, is like a barbecue master who uses high heat (a laser) to fuse together powder materials.\n\n3. Materials Used in 3D Printing: Just like how you choose different 

**Sequential Chains**

In [22]:
# This is an LLMChain to write a synopsis given a title of a play.
# llm = ChatOpenAI(temperature=.7)
template = """You are a learning assistant. Given a technical subject, write down 5 fundamental concepts to understand it.
Subject: {subject}
Field: {field}
Learning assistant: The 5 fundamental concepts are:"""
prompt_template = PromptTemplate(input_variables=["subject", "field"], template=template)
learning_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="concepts")

In [23]:
# This is an LLMChain to write a review of a play given a synopsis.
# llm = ChatOpenAI(temperature=.7)
template = """You are an expert teacher in all technical and scientific fields. Given a list of 5 concepts, write down a simple intuitive explanation of each concept.

Concepts:
{concepts}
Intuitive explanations:"""
prompt_template = PromptTemplate(input_variables=["concepts"], template=template)
explanation_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="explanations")

In [24]:
# This is the overall chain where we run these two chains in sequence.
learning_overall_chain = SequentialChain(
    chains=[learning_chain, explanation_chain],
    input_variables=["subject", "field"],
    # Here we return multiple variables
    output_variables=["concepts", "explanations"],
    verbose=True)

In [25]:
learning_overall_chain({"subject":"tennis", "field": "engineering"})



> Entering new SequentialChain chain...

> Finished chain.


{'subject': 'tennis',
 'field': 'engineering',
 'concepts': '1. Physics of Tennis: Understanding the physics involved in the sport, including the principles of force, motion, and energy transfer, which are crucial in serving, hitting, and returning the ball effectively.\n\n2. Material Science: Knowledge about different materials used in tennis equipment like racquets and balls and how their properties affect their performance. \n\n3. Aerodynamics: Understanding how the shape and spin of the ball can influence its trajectory and speed, and how these factors can be manipulated for strategic advantage.\n\n4. Biomechanics: Understanding the mechanics of the human body, particularly in relation to movement and exertion during a tennis match. This includes the study of muscle groups, force exerted, and injury prevention.\n\n5. Engineering Design: Knowledge about the engineering principles involved in designing and improving tennis equipment and facilities, such as racquets, courts, and train

# Simple Q&A Example

In [36]:
#!pip install docarray
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [37]:
import pandas as pd
df = pd.read_csv("./superheroes.csv")
df.head()

,Superhero Name,Superpower,Power Level,Catchphrase
0,Captain Thunder,Bolt Manipulation,90,Feel the power of the storm!
1,Silver Falcon,Flight and Agility,85,"Soar high, fearlessly!"
2,Mystic Shadow,Invisibility and Illusions,78,Disappear into the darkness!
3,Blaze Runner,Pyrokinesis,88,Burn bright and fierce!
4,Electra-Wave,Electric Manipulation,82,Unleash the electric waves!


In [33]:
file = 'superheroes.csv'
loader = CSVLoader(file_path=file)

Now, let's set up our Vector store (we'll talk about what that is in a second):

In [34]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])

In [62]:
query = "Tell me the catch phrase for Captain Thunder"

In [63]:
response = index.query(query)

In [64]:
display(Markdown(response))

 Captain Thunder's catchphrase is "Feel the power of the storm!"

Ok, cool! So, now let's backtrack a little bit and discuss what is going on.

If we want LLMs to get access to our data in order to help us get insights, we have one major problem: LLMs have a limited context window, meaning they can only process a few thousand words at a time which constraints their ability to answer questions (for example) of really big documents. 

That's where things like embeddings and vector stores come into play, these are way of representing the data in such a way to facilitate the access by an LLM. Let's break it down, starting with embeddings:

Embeddings are numerical representations of data, meaning, their a way to represent data such that the semantic information of that data is properly reflected in the distances between the data points in the embedding.

That means that text with similar content will have similar vectors (which is a way of saying that they'll be closer together in the embedding).

![](2023-07-30-19-29-27.png)

[LangChain for LLM Application Development by Deeplearning.ai](https://learn.deeplearning.ai/langchain/lesson/1/introduction)

Now, let's talk about vector databases.

A vector database is a way to store these embeddings, these numerical representations that we just discussed.

The pipeline is:
- In coming document
- Create chunks of text from that document
- Embed each chunk
- Store these embeddings

![](2023-07-30-19-32-13.png)

[LangChain for LLM Application Development by Deeplearning.ai](https://learn.deeplearning.ai/langchain/lesson/1/introduction)

Now, we can query those embeddings stored in the vector database to get the most relevant responses!

So when we create a query, that query is first embedded and then we compare its embedding with the embeddings we have stored in the vector database. We then select the N-most similar embeddings, and pass those to the LLM.

![](images/2023-07-30-19-34-48.png)

[LangChain for LLM Application Development by Deeplearning.ai](https://learn.deeplearning.ai/langchain/lesson/1/introduction)

- __Embedding__: the useful data representation that will be used as the thing (or things) we can query
- __Vector Database__: the vectorization of the embedding chunks (the database for the embeddings where we can do the query)

### Embeddings


![](./images/embeddings.png)

![](./images/embeddings2.png)


### Vector Database




![](./images/2023-07-17-12-48-28.png)



# References
- https://python.langchain.com/docs/get_started/introduction.html
- https://medium.com/@remitoffoli/a-visual-guide-to-llm-powered-app-architecture-57e47426a92f
- [LangChain for LLM App Development short course by coursera](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)
- [LLM Evaluation](https://learn.deeplearning.ai/langchain/lesson/6/evaluation)
[Models, Prompts, parsers, memory and chains from this langchain for](https://learn.deeplearning.ai/langchain/lesson/7/agents)
- [Chat With Your Data - Retrieval](https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/5/retrieval)
- [Emebeddings simple definition](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)
- [Vector DBs - simple definition](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)